In [3]:
# import modules and settings
import pandas as pd
import numpy as np
import glob
pd.options.display.max_colwidth =1000



In [4]:
# combine all the raw data to one single dataframe
path = r'C:\Users\avci\Documents\MSE\HS20\VT1\aimu.v1.0\aimu.v1.0'
all_files = glob.glob(path + "/*.trans")
li = []

for filename in all_files:
    df = pd.read_csv(filename, sep='\t', names = ['txtID','txtStart','txtStop', 'speakerID','utterance' ])
    li.append(df)

rawdf = pd.concat(li, axis=0, ignore_index=True)

In [5]:
def catch_ai_type(df):
    
    '''
    1: creates a binary label actionItem if  occurs inutterance
    2: assigns an action item type and subtype for each utterance
    '''
    
    # binary labeling
    df["actionItem"]= df["utterance"].str.find("<") 
    df['actionItem'] = [False if x < 0 else True for x in df['actionItem']]
    
    # multiclas labeling
    df['aitype'] = 'noAI'
    df['aisubtype'] = 'noAI'
    
    for i, utterance in enumerate(df['utterance']):
        #Reminders
        if utterance.find('<create_single_reminder>') != -1:
            df.at[i, 'aitype'] = str('Reminder')
            df.at[i, 'aisubtype'] = str('create_single_reminder')
         
        #Calendar
        if utterance.find('<find_calendar_entry>') != -1:
            df.at[i, 'aitype'] = str('Calendar')
            df.at[i, 'aisubtype'] = str('find_calendar_entry')

        if utterance.find('<create_calendar_entry>') != -1:
            df.at[i, 'aitype'] = str('Calendar')
            df.at[i, 'aisubtype'] = str('create_calendar_entry')

        if utterance.find('<add_agenda_item>') != -1:
            df.at[i, 'aitype'] = str('Calendar')
            df.at[i, 'aisubtype'] = str('add_agenda_item')
            
        if utterance.find('<open_agenda>') != -1:
            df.at[i, 'aitype'] = str('Calendar')
            df.at[i, 'aisubtype'] = str('open_agenda')
            
        #Communication           
        if utterance.find('<send_email>') != -1:
            df.at[i, 'aitype'] = str('Communication')
            df.at[i, 'aisubtype'] = str('send_email')

        if utterance.find('<find_email>') != -1:
            df.at[i, 'aitype'] = str('Communication')
            df.at[i, 'aisubtype'] = str('find_email')

        if utterance.find('<make_call>') != -1:
            df.at[i, 'aitype'] = str('Communication')
            df.at[i, 'aisubtype'] = str('make_call')

        #Search
        if utterance.find('<search>') != -1:
            df.at[i, 'aitype'] = str('search')
            df.at[i, 'aisubtype'] = str('search')
            
        #OnDevice
        if utterance.find('<open_setting>') != -1:
            df.at[i, 'aitype'] = str('OnDevice')
            df.at[i, 'aisubtype'] = str('OnDevice')
    return df

            
            
            
dftagged = catch_ai_type(df = rawdf)       

In [6]:
def catch_ai_tag(df):
    
    '''
    1: searches for ai tags if a ai type is found
    2: saves ai tags with corresponding tags in df
    '''
    
    for i, utterance in enumerate(df['utterance']):
        #Reminders
        if utterance.find('<create_single_reminder>') != -1:
            for arg in ['contact_name','reminder_text', 'start_date', 'start_time']:
                if str(utterance).find('<'  + arg + '>') != -1:
                    n = len(arg)+2
                    st = str(utterance).find('<'  + arg + '>')
                    en = str(utterance).find('</' + arg + '>')
                    obs = utterance[st+n:en]
                    df.at[i, arg] = str(obs)
                    
                if str(utterance).find('<'  + arg + '>') == -1:
                    df.at[i,arg] = 'NA'
                    
        #Calendar
        if any(x in utterance for x in 
               ['<find_calendar_entry>','<create_calendar_entry>','<add_agenda_item>','<open_agenda>']):
            for arg in ['contact_name','entry_type', 'start_date','end_date', 'start_time','end_time',
                        'title','absolute_location','implicit_location', 'agenda_item']:
                if str(utterance).find('<'  + arg + '>') != -1:
                    n = len(arg)+2
                    st = str(utterance).find('<'  + arg + '>')
                    en = str(utterance).find('</' + arg + '>')
                    obs = str(utterance[st+n:en])
                    df.at[i, arg] = str(obs)
                    
                if str(utterance).find('<'  + arg + '>') == -1:
                    df.at[i,arg] = 'NA'
        
        
        #Communication
        if any(x in utterance for x in 
               ['<send_email>','<find_email>','<make_call>','<open_agenda>']):
            for arg in ['contact_name','email_subject', 'email_content','from_contact_name']:
                if str(utterance).find('<'  + arg + '>') != -1:
                    n = len(arg)+2
                    st = str(utterance).find('<'  + arg + '>')
                    en = str(utterance).find('</' + arg + '>')
                    obs = str(utterance[st+n:en])
                    df.at[i, arg] = str(obs)
                    
                if str(utterance).find('<'  + arg + '>') == -1:
                    df.at[i,arg] = 'NA'        

        #Search
        if utterance.find('<search>') != -1:
            for arg in ['query_term']:
                if str(utterance).find('<'  + arg + '>') != -1:
                    n = len(arg)+2
                    st = str(utterance).find('<'  + arg + '>')
                    en = str(utterance).find('</' + arg + '>')
                    obs = utterance[st+n:en]
                    df.at[i, arg] = str(obs)
                    
                if str(utterance).find('<'  + arg + '>') == -1:
                    df.at[i,arg] = 'NA'
            
        #ondevice
        if utterance.find('<open_setting>') != -1:
            for arg in ['setting_type']:
                if str(utterance).find('<'  + arg + '>') != -1:
                    n = len(arg)+2
                    st = str(utterance).find('<'  + arg + '>')
                    en = str(utterance).find('</' + arg + '>')
                    obs = utterance[st+n:en]
                    df.at[i, arg] = str(obs)
                    
                if str(utterance).find('<'  + arg + '>') == -1:
                    df.at[i,arg] = 'NA'
                    
    return df
       
    
    
    
dftagged = catch_ai_tag(df = dftagged)       

dfai = dftagged[dftagged['actionItem'] == True]


In [11]:
data = dftagged[['utterance', 'actionItem']]


In [12]:
data.to_csv('data.txt', header=True, index=False, sep='\t', mode='a')


In [17]:
file = open('data.txt', 'w+')
for i in data.index:
    line = '_label_' + str(data['actionItem'][i]) + ' ' + data['utterance'][i]
    file.write(line + '\n')
file.close()

In [ ]:
! pip install fasttext
